In [172]:
from __future__ import print_function
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from itertools import chain
from elasticsearch_dsl import Q
from elasticsearch_dsl.query import MoreLikeThis
import json
import pandas as pd
import numpy as np

In [173]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "YhF4vfe",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "CPv0HmFPQvChtynShF0b5A",\n  "version" : {\n    "number" : "5.6.0",\n    "build_hash" : "781a835",\n    "build_date" : "2017-09-07T03:09:58.087Z",\n    "build_snapshot" : false,\n    "lucene_version" : "6.6.0"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [174]:
def scrollr(client, scroll_id, f):
    r =  client.scroll(scroll_id, scroll="2m")
    while len(r['hits']['hits']) > 0:
        for item in r['hits']['hits']:
            yield f(item)
        r = client.scroll(scroll_id, scroll="2m")

In [175]:
def extract_references(article):
    return article['_id']

In [176]:
index ='pangaea-recommender'
index_type='rec'
host ='http://localhost:9200'

In [177]:
doc = {'query': {'match_all' : {}}}

In [178]:
listall=[]
client = Elasticsearch(host)
r = client.search(index, index_type, body=doc, scroll="2m")
scroll_id = r['_scroll_id']
for ref in scrollr(client, scroll_id, extract_references):
    listall.append(int(ref))
    
(len(listall)), len(set(listall))

(319364, 319364)

In [179]:
listall[:3]

[50027, 50028, 50029]

# ES - Query Terms 1,2

In [180]:
rows_list = []
size =5
client = Elasticsearch(host)
for dId in listall:
    q1 = Q(MoreLikeThis(like={'_id': dId}, fields=['query_1'],min_term_freq=0,min_doc_freq=5,max_query_terms=25,boost_terms=1, boost=2, minimum_should_match="10%"))
    q2 = Q(MoreLikeThis(like={'_id': dId}, fields=['query_2'],min_term_freq=0,min_doc_freq=5,max_query_terms=25,boost_terms=1, boost=1, minimum_should_match="10%"))
    q = Q('bool', should=[q1,q2])
    s = Search().using(client)
    s = s.query(q)
    s = s[0:size]
    #print(json.dumps(s.to_dict()))
    response = s.execute()
    #print(response.hits.total)
    d = {}
    d['_id'] = dId
    l = []
    for row in response['hits']['hits']:
        l.append(row['_id'])
    d['related-by-query'] = l
    rows_list.append(d)

In [181]:
df = pd.DataFrame(rows_list) 
df.head()

,_id,related-by-query
0,50027,"[50026, 815911, 143479, 614753, 57296]"
1,50028,[]
2,50029,"[734017, 602307, 267774, 734373, 54866]"
3,50030,[]
4,50031,"[711067, 138308, 228648, 50166, 51669]"


In [182]:
df.shape

(319364, 2)

In [184]:
import os
parent_dir = os.path.dirname(os.path.dirname(os.path.abspath(os.path.realpath('__file__'))))
df.to_csv(os.path.abspath(os.path.join(parent_dir, 'results'))+'/es_relbyqueries.csv', sep='\t', encoding='utf-8')